In [1]:
# Connect to and Instantiate Fuseki DB
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from rdflib import URIRef, Literal, Graph, Dataset
    
import db_admin
from datetime import datetime

In [2]:
import pandas as pd
import html
import loader
import discourse

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/tomk/environment/devpy/lib/python3.10/site-packages/owlready2/hermit:/home/tomk/environment/devpy/lib/python3.10/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmp2cg9rtu4


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.49588918685913086 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [3]:
DS = db_admin.dataset_handle("http://localhost:3030/modelg/query", "http://localhost:3030/modelg/update")

In [4]:
namespaces = [('xml' , "http://www.w3.org/XML/1998/namespace"), 
               ('rdf',"http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
          ('dc',"http://purl.org/dc/elements/1.1/"),
          ('rdfs',"http://www.w3.org/2000/01/rdf-schema#")
         ]

properties=g_context = { "dc:created" : datetime.now().strftime("%Y-%m-%d"),
              "dc:description":"""The master graph is used to store all mastered triples. It acts as a "soup" of expressable content, primarily for entities to be identified by their properties for the purposes of identifying unique individuals. """}


print(db_admin.create_named_graph_manifest_rdf_xml("http://master", namespaces, properties))

<rdf:RDF xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
	<rdf:Description rdf:about="http://master">
		<dc:created>2023-06-29</dc:created>
		<dc:description>The master graph is used to store all mastered triples. It acts as a "soup" of expressable content, primarily for entities to be identified by their properties for the purposes of identifying unique individuals. </dc:description>
	</rdf:Description>
</rdf:RDF>


In [5]:
files = db_admin.read_sparql_repo("../SPARQL")

.sparql
.sparql


In [6]:
import re

manifests={}
for f in files:
    gs = db_admin.read_sparql_manifest(f)
    if 'title' in gs:
        manifests[gs['title']]=gs


In [7]:
manifests['get_named_graphs']['query']

'select distinct ?g WHERE { GRAPH ?g { ?s ?p ?o .}}'

In [8]:
rs = DS.query(manifests['get_named_graphs']['query'])
pd.DataFrame(rs)

,0
0,http://config
1,http://discourse
2,http://master


In [9]:
g_uris_d = { "config" : ("http://config", { "dc:created" : datetime.now().strftime("%Y-%m-%d"),
                                           "dc:description":"""The config graph is used to host models and Serialization configuration defining content within the wider graph. """}),
             "master" : ("http://master",{ "dc:created" : datetime.now().strftime("%Y-%m-%d"),
                                           "dc:description":"""The master graph is used to store all mastered triples. It acts as a "soup" of expressable content, primarily for entities to be identified by their properties for the purposes of identifying unique individuals. """}),
             "discourse" : ("http://discourse", { "dc:created" : datetime.now().strftime("%Y-%m-%d"),
                                           "dc:description":"""The discourse graph is used to manage discourses, that is, statements and reports loaded into the system in a form that allows them to be compared, even where they may contradict. """}), }


for k,data in g_uris_d.items():
    v, properties = data
    gmanifest=db_admin.create_named_graph_manifest_rdf_xml(v, namespaces, properties)
    gmanifest=gmanifest
    working_g = DS.graph(URIRef(v))
    working_g.parse(data=gmanifest, format="application/rdf+xml")

In [10]:
# We need to populate the config graph with some contents from an rdf file, if it's not already populated
# This method is incredibly slow, but (should) only be necessary on the occasions where the database
# is being populated from scratch. There are probably better ways to insert contents of a graph into the db
# and this might be something to revisit later on. 
sg = DS.graph(URIRef(g_uris_d["config"][0]))
sg.parse("DMEAR_ser.rdf")
#S = serialization.Serialization(sg, "EntityAttributeRecord")

<Graph identifier=http://config (<class 'rdflib.graph.Graph'>)>

In [11]:
print(gmanifest)

<rdf:RDF xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
	<rdf:Description rdf:about="http://discourse">
		<dc:created>2023-06-29</dc:created>
		<dc:description>The discourse graph is used to manage discourses, that is, statements and reports loaded into the system in a form that allows them to be compared, even where they may contradict. </dc:description>
	</rdf:Description>
</rdf:RDF>


In [12]:
rs = DS.query(manifests['get_named_graphs']['query'])
pd.DataFrame(rs)

,0
0,http://config
1,http://discourse
2,http://master


In [13]:
rs = DS.query('select ?g ?p ?o WHERE { GRAPH ?g { ?g ?p ?o .}}')
pd.DataFrame(rs)

,0,1,2
0,http://config,http://purl.org/dc/elements/1.1/created,2023-06-29
1,http://config,http://purl.org/dc/elements/1.1/description,The config graph is used to host models and Se...
2,http://discourse,http://purl.org/dc/elements/1.1/created,2023-06-29
3,http://discourse,http://purl.org/dc/elements/1.1/description,The discourse graph is used to manage discours...
4,http://master,http://purl.org/dc/elements/1.1/created,2023-06-29
5,http://master,http://purl.org/dc/elements/1.1/description,The master graph is used to store all mastered...


In [14]:
discourse_details_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?name ?title ?description ?created ?hash (COUNT(?declaration) as ?declarations) (COUNT(distinct ?psubject) as ?entities)

WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse dcterms:title ?title.
      ?discourse rdfs:label ?name.
      ?discourse dcterms:description ?description.
      ?discourse dcterms:created ?created .
      ?discourse disco:DiscourseHash ?hash .
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
      ?posit_type rdfs:subPropertyOf* disco:Posits .
      ?declaration disco:Asserts ?assertion.
      ?assertion disco:Subject ?psubject.
  }
}
GROUP BY ?g ?discourse ?name ?title ?description ?created ?hash
"""

qr = DS.query(discourse_details_sparql)
discourse_df = pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="created")
discourse_df["hash_vals"]=discourse_df['hash'].apply(lambda x : x.n3()[1:-1])
discourse_df.set_index("discourse", inplace=True)
discourse_df
hash_discourse_mapping = {v:k for k,v in dict(discourse_df['hash'].apply(lambda x : x.n3()[1:-1])).items()}
hash_discourse_mapping

{'9c6b3c3a52c5-a373-0c14-5a07a1419d17': rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/2173357ae21240129013aca6eb89c504')}

In [15]:
file_dtypes = {'ModelDomain': str,
 'Model': str,
 'ModelType': str,
 'Class': str,
 'ClassDescription': str,
 'Attribute': str,
 'AttributeDescription': str,
 'DataType': str,
 'Context': str,
 'SubContext': str,
 'Nulls': str,
 'Is PK': str,
 'Relationship': str,
 'FromClass': str,
 'FromAttribute': str,
 'FromCardinality': str,
 'ToClass': str,
 'ToAttribute': str,
 'ToCardinality': str}

In [16]:
def get_field(v):
    if isinstance(v,str):
        return html.escape(str(v))
    elif isinstance(v,(int, float)):
        if pd.isnull(v):
            return None
        else:
            return v
    elif v is None or isinstance(v,pd.Null):
        return None

In [17]:
def meta_data_package_template(field_d):
    dc_terms_base = "http://purl.org/dc/terms/"
    rdf_form = {}
    for k,v in field_d.items():
        rdf_form[URIRef(dc_terms_base + k)] = Literal(v)
    return rdf_form


def triples_to_quads(triples, graph="http://master"):
    for s,p,o, *_ in triples:
        yield (s,p,o,URIRef(graph))

In [18]:
title = "Inventory ERD v1.0"
model_data = pd.read_csv(r"../sample/inventory_v1.csv", index_col="Sequence", dtype=file_dtypes)
rows = [dict({rk:get_field(rv) for rk, rv in r.items()}) for i,r in model_data.iterrows()]
md_p = meta_data_package_template( {"created" : "2023-04-10", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 1", 
                                   "title" : "inventory ERD", 
                                   "modified" : "2023-04-10"})

loader.load_to_graph(DS, g_uris_d["config"][0], "EntityAttributeRecord", rows, g_uris_d["master"][0], g_uris_d["discourse"][0], title, md_p, hash_discourse_mapping, override_duplicate=False)



0:00:00.125759 for 9
self.members: 75
This data is already loaded! - Aborting


In [19]:
g_uris_d["discourse"][0]

'http://discourse'